In [1]:
#Witting mechanism generation 23112024@0932

In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdChemReactions as Reactions
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
import matplotlib.pyplot as plt
from rdkit.Chem import rdDepictor
rdDepictor.SetPreferCoordGen(True)
import os
os.getcwd()

import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions 
import re
from collections import Counter
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdChemReactions
from rdkit.Chem import rdDepictor
rdDepictor.SetPreferCoordGen(True)
from rdchiral.template_extractor import extract_from_reaction

In [3]:
def remove_number(smiles):
    mol = Chem.MolFromSmiles(smiles)
    [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
    return Chem.MolToSmiles(mol)

#this code is from:
def make_rxns(source_rxn, reactants):
    product_sets = source_rxn.RunReactants(reactants)
    new_rxns = []
    for pset in product_sets:
        new_rxn = AllChem.ChemicalReaction()
        for react in reactants:
            react = Chem.Mol(react)
            for a in react.GetAtoms():
                a.SetIntProp('molAtomMapNumber', a.GetIdx())
            new_rxn.AddReactantTemplate(react)
        for prod in pset:
            for a in prod.GetAtoms():
                a.SetIntProp('molAtomMapNumber', int(a.GetProp('react_atom_idx')))
            new_rxn.AddProductTemplate(prod)
        new_rxns.append(new_rxn)
    return new_rxns

def moveAtomMapsToNotes(m):
    for at in m.GetAtoms():
        if at.GetAtomMapNum():
            at.SetProp("atomNote",str(at.GetAtomMapNum()))
            #at.SetAtomMapNum(0)

def RxnTemplateExtraction(reaction):
    '''given a reaction smiles, it returns
    a reaction template using rdchiral tool
    input: A.B>C.D>E.F'''
    react_dict = {}
    react_splitted = reaction.split('>')
    react_dict['_id'] = 1
    react_dict['reactants'] = react_splitted[2]
    react_dict['products'] = react_splitted[0]
    rxn_template = extract_from_reaction(react_dict)
    return rxn_template['reaction_smarts']

In [4]:
def next_rxn(input_rxn_smiles):
    rxn_splits = input_rxn_smiles.split('>')
    product_reagents = '.'.join([rxn_splits[1], rxn_splits[2]])
    return product_reagents

def RxnToTemplates(rxn_dict):
    '''input: atom mapped reactions as dictionary, key:value pair'''
    templates_ = []
    for i in rxn_dict:
        templates_.append(RxnTemplateExtraction(rxn_dict[i]))
    templates = []
    for i in templates_:
        parenthesis_id = i.index('>')
        templates.append('(' + i[:parenthesis_id] + ')' + i[parenthesis_id:])
    return templates

from rdkit import Chem
from collections import Counter

def increase_mapping_numbers(reaction):
    # Define a regular expression pattern to match the atom mapping numbers
    pattern = r'(?<=:)(\d+)'

    # Define a function to increase each matched number by one
    def increase(match):
        return str(int(match.group(0)) + 1)

    # Use re.sub() to replace the matched numbers with the increased values
    modified_reaction = re.sub(pattern, increase, reaction)

    return modified_reaction

def remove_ids(smile):
    mol = Chem.MolFromSmiles(smile)
    [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
    return Chem.MolToSmiles(mol)

def remove_leading_dot(s):
    return s.lstrip('.') if s.startswith('.') else s

def rxn_format_change(rxn):
    reactant, reagent, product = rxn.split('>')
    reagent = remove_leading_dot(reagent) #sometime it contain extra '.' at the begining, so I am remvoing it
    reagent_wo_ids = remove_ids(reagent)
    reactant_reagent = reactant if not reagent_wo_ids else f"{reactant}.{reagent_wo_ids}"
    new_rxn = '>>'.join((reactant_reagent, product))
    return new_rxn

def mol_can(smi):
    mol = Chem.MolFromSmiles(smi)
    [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
    return Chem.MolToSmiles(mol)

import pandas as pd


def remove_ids(smile):
    mol = Chem.MolFromSmiles(smile)
    [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
    return Chem.MolToSmiles(mol)

def remove_leading_dot(s):
    return s.lstrip('.') if s.startswith('.') else s

def rxn_format_change(rxn):
    try:
        rxn = increase_mapping_numbers(rxn)
        reactant, reagent, product = rxn.split('>')
        reagent = remove_leading_dot(reagent) #sometime it contain extra '.' at the begining, so I am remvoing it
        reagent_wo_ids = remove_ids(reagent)
        reactant_reagent = reactant if not reagent_wo_ids else f"{reactant}.{reagent_wo_ids}"
        new_rxn = '>>'.join((reactant_reagent, product))
    except:
        new_rxn = rxn
    return new_rxn

def rxn_change(all_mech):
    x = []
    for i in all_mech:
        x.append(rxn_format_change(i))
    return x

In [5]:
from itertools import permutations
from rdkit import Chem
from collections import Counter

def match_pdts(true_pdts, gen_pdts):
    # Split each input string into separate SMILES components
    true_parts = true_pdts.split('.')
    gen_parts = gen_pdts.split('.')
    # Canonicalize each component of true_pdts
    true_canonicals = set()
    for smi in true_parts:
        mol = Chem.MolFromSmiles(smi)
        [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
        if mol is not None:
            # Convert to canonical SMILES without stereochemistry
            true_canonicals.add(Chem.MolToSmiles(mol, isomericSmiles=False))
    
    # Canonicalize and compare each component of gen_pdts
    for smi in gen_parts:
        mol = Chem.MolFromSmiles(smi)
        if mol is not None:
            [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
            gen_canonical = Chem.MolToSmiles(mol, isomericSmiles=False)
            if gen_canonical in true_canonicals:
                return True
    
    return False

def find_all_reac_reag_comb(template_smarts_str, str_of_smiles):
    #print('these are getting used:', template_smarts_str, str_of_smiles)
    list_of_smiles = str_of_smiles.split('.')
    #print('all smiels:', list_of_smiles)
    reactants_smarts = template_smarts_str.split('>>')[0][1:-1]
    reactants_smarts = reactants_smarts.split('.')
    #print('reactants_smarts:', reactants_smarts)
    # Dictionary to hold matches for each SMARTS pattern
    smarts_to_matches = {smarts: [] for smarts in reactants_smarts}
    
    for smiles in list_of_smiles:
        mol = Chem.MolFromSmiles(smiles)
        for smarts in reactants_smarts:
            smarts_mol = Chem.MolFromSmarts(smarts)
            if mol.HasSubstructMatch(smarts_mol):
                smarts_to_matches[smarts].append(smiles)
    
    # Generate all permutations of matches for each SMARTS pattern
    reactant_combinations = []
    for reactant_permutation in permutations(list_of_smiles, len(reactants_smarts)):
        match = True
        for reactant, smarts in zip(reactant_permutation, reactants_smarts):
            mol = Chem.MolFromSmiles(reactant)
            smarts_mol = Chem.MolFromSmarts(smarts)
            if not mol.HasSubstructMatch(smarts_mol):
                match = False
                break
        if match:
            reactant_combinations.append(list(reactant_permutation))
    
    
    # Remove duplicate combinations
    unique_combinations = []
    for combo in reactant_combinations:
        if combo not in unique_combinations:
            unique_combinations.append(combo)
    
    # Generate reagents for each reactant combination, maintaining counts
    results = []
    original_counts = Counter(list_of_smiles)  # Keep track of the original counts
    for reactants in unique_combinations:
        reactant_counts = Counter(reactants)
        reagents_counts = original_counts - reactant_counts  # Subtract reactant counts
        reagents = list(reagents_counts.elements())  # Reconstruct the reagents list
        results.append((reactants, reagents))

    
    return results

def transform_func(template_smarts_str, str_of_smiles):
    #reactants_smiles_list, reagents_smiles_list = reactant_reagent_finder(template_smarts_str, str_of_smiles)
    #print('\ninput to find_all_reac_reag_comb:', template_smarts_str,str_of_smiles)
    reactant_reagents = find_all_reac_reag_comb(template_smarts_str,str_of_smiles)
    if not reactant_reagents:
        print(f"No valid reactant-reagent combinations found for: {template_smarts_str} with {str_of_smiles}")
        return []  # Return an empty list or handle appropriately
    
    #print('\noutput reactant_reagents', reactant_reagents)
    reactants_smiles_list = reactant_reagents[0][0]
    reagents_smiles_list = reactant_reagents[0][1]
    
    #print(reactants_smiles_list, reagents_smiles_list)
    reagent_smiles = '.'.join(reagents_smiles_list)
    reactants_mols_list = [Chem.MolFromSmiles(smiles) for smiles in reactants_smiles_list]
    template = AllChem.ReactionFromSmarts(template_smarts_str)
    
    #-------------------combine the reactants into a single mol object--------------
    rmol = None
    for mol in reactants_mols_list:
        if rmol is None:
            rmol = mol  # If rmol is None, set it to the first molecule
        else:
            rmol = Chem.CombineMols(rmol, mol)
    #----------------------mapping the reactants and products-------------------------
    atom_mapped_rxns = []
    for r in make_rxns(template, [rmol]):
        smi = AllChem.ReactionToSmiles(r)
        smi =  re.sub(r'^\((.*)\)>',r'\1>',smi) #this is required
        atom_mapped_rxns.append(smi)
        
    #print('atom mapped rxns:', atom_mapped_rxns)
    #--------------------------adding the reagents------------------------------------
    mapped_rxns = []
    for i in atom_mapped_rxns:
        mapped_rxns.append(i.replace(">>", f">{reagent_smiles}>"))
    mapped_rxns = list(set(mapped_rxns))
    #print('mapped_rxns:', mapped_rxns)
    return mapped_rxns


def dfs_with_processing1(start, transform_func, validate_func, target_output, param_list):
    visited = set()  # To track visited states and avoid cycles

    def process_output(output):
        """Custom processing logic on the outputs if required."""
        try:
            output_reagent = output.split('>')[1]
            output_pdt = output.split('>')[2]
            if output_reagent:
                processed = '.'.join([output_pdt, output_reagent])
            else:
                processed = output_pdt
            return processed
        
        except IndexError:
            return None  # If processing fails, return None to signal an invalid path

    def is_valid_molecule(smiles):
        """Check if a SMILES string corresponds to a valid molecule."""
        try:
            mol = Chem.MolFromSmiles(smiles)
            return mol is not None
        except:
            return False

    def dfs(current_input, path, param_index):
        # Stop if the current input matches the target output
        if validate_func(current_input, target_output):
            print('Got it, matched!')
            return path

        if (current_input, param_index) in visited:
            return None
        visited.add((current_input, param_index))

        # Check if we've exhausted all transformation rules
        if param_index >= len(param_list):
            return None

        # Apply the current transformation rule
        current_param = param_list[param_index]
        raw_outputs = transform_func(current_param, current_input)  # Raw outputs from transform_func  
        processed_outputs = []

        for raw in raw_outputs:
            # Process the raw output
            processed = process_output(raw)
            if processed is None:
                continue  # Skip invalid processing results

            # Validate individual SMILES in the processed output
            components = processed.split('.')
            if all(is_valid_molecule(comp) for comp in components):
                processed_outputs.append((raw, processed))
        
        for raw_output, processed_output in processed_outputs:
            # Recursive DFS with the next SMARTS pattern in param_list
            result = dfs(processed_output, path + [(current_param, raw_output, processed_output)], param_index + 1)
            if result is not None:
                return result

        return None  # No valid path found

    # Start with the first SMARTS pattern
    return dfs(start, [], 0)


In [6]:
template_smarts_str = '([CH2:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH-:4])>>[CH-:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH2;+0:4]'
str_of_smiles = 'O=C1CNC(N1)=O.O=Cc2c3nc(NC4C(CCC4=O)=O)cc(NC5CC5)n3nc2.[OH-]'

results = find_all_reac_reag_comb(template_smarts_str, str_of_smiles)
for reactants, reagents in results:
    print(f"Reactants: {reactants}, Reagents: {reagents}")


Reactants: ['O=C1CNC(N1)=O', '[OH-]'], Reagents: ['O=Cc2c3nc(NC4C(CCC4=O)=O)cc(NC5CC5)n3nc2']
Reactants: ['O=Cc2c3nc(NC4C(CCC4=O)=O)cc(NC5CC5)n3nc2', '[OH-]'], Reagents: ['O=C1CNC(N1)=O']


In [7]:
param_list = ['([CH2:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH-:4])>>[CH-:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH2;+0:4]',
              '([CH-:1]-[C;H0;D3;+0:2]=[O:3].[O:4]=[C;H1;D2;+0:5][*;H0;+0:6])>>[O;-1:4]-[C;H1;D3;+0:5](-[*;H0;+0:6])-[CH;+0:1]-[C;H0;D3;+0:2]=[O:3]']


In [8]:
start = 'O=C1CC(CC1(C)C)(C)C.O=CC2=CC(C(C)(C)C)=C(C=C2)C=O.[OH-]'
target_output = 'O=C1C(C(C)(CC1)C)C([O-])C2=CC=C(C=C2)C=O'
dfs_with_processing1(start, transform_func, match_pdts, target_output, param_list)

In [9]:
start = 'O=C1CC(CC1(C)C)(C)C.O=CC2=CC(C(C)(C)C)=C(C=C2)C=O.[OH-]'
target_output = 'O=C1C(C(CC1(C)C)(C)C)C([O-])C2=CC(C(C)(C)C)=C(C=C2)C=O'
dfs_with_processing1(start, transform_func, match_pdts, target_output, param_list)

Got it, matched!


[('([CH2:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH-:4])>>[CH-:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH2;+0:4]',
  '[O:0]=[C:1]1[CH2:2][C:3]([CH3:8])([CH3:9])[CH2:4][C:5]1([CH3:6])[CH3:7].[OH-:10]>O=CC2=CC(C(C)(C)C)=C(C=C2)C=O>[O:0]=[C:1]1[CH-:2][C:3]([CH3:8])([CH3:9])[CH2:4][C:5]1([CH3:6])[CH3:7].[OH2:10]',
  '[O:0]=[C:1]1[CH-:2][C:3]([CH3:8])([CH3:9])[CH2:4][C:5]1([CH3:6])[CH3:7].[OH2:10].O=CC2=CC(C(C)(C)C)=C(C=C2)C=O'),
 ('([CH-:1]-[C;H0;D3;+0:2]=[O:3].[O:4]=[C;H1;D2;+0:5][*;H0;+0:6])>>[O;-1:4]-[C;H1;D3;+0:5](-[*;H0;+0:6])-[CH;+0:1]-[C;H0;D3;+0:2]=[O:3]',
  '[O:0]=[C:1]1[CH-:2][C:3]([CH3:4])([CH3:5])[CH2:6][C:7]1([CH3:8])[CH3:9].[O:10]=[CH:11][c:12]1[cH:13][c:14]([C:15]([CH3:16])([CH3:17])[CH3:18])[c:19]([CH:22]=[O:23])[cH:20][cH:21]1>[OH2:10]>[O:0]=[C:1]1[CH:2]([CH:11]([O-:10])[c:12]2[cH:13][c:14]([C:15]([CH3:16])([CH3:17])[CH3:18])[c:19]([CH:22]=[O:23])[cH:20][cH:21]2)[C:3]([CH3:4])([CH3:5])[CH2:6][C:7]1([CH3:8])[CH3:9]',
  '[O:0]=[C:1]1[CH:2]([CH:11]([O-:10])[c:12]2[cH:13][c:14]([C:15]([CH3:

In [10]:
start = 'O=C1CC(CC1(C)C)(C(CC)=O)C.O=CC2=CC(C(C)(C)C)=C(C=C2)C=O.[OH-]'
target_output = 'O=C1C(C(CC1(C)C)(C)C(CC)=O)C(C(C=CC(C=O)=C2)=C2C(C)(C)C)[O-]'
dfs_with_processing1(start, transform_func, match_pdts, target_output, param_list)

Got it, matched!


[('([CH2:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH-:4])>>[CH-:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH2;+0:4]',
  '[O:0]=[C:1]1[CH2:2][C:3]([C:8]([CH2:9][CH3:10])=[O:11])([CH3:12])[CH2:4][C:5]1([CH3:6])[CH3:7].[OH-:13]>O=CC2=CC(C(C)(C)C)=C(C=C2)C=O>[O:0]=[C:1]1[CH-:2][C:3]([C:8]([CH2:9][CH3:10])=[O:11])([CH3:12])[CH2:4][C:5]1([CH3:6])[CH3:7].[OH2:13]',
  '[O:0]=[C:1]1[CH-:2][C:3]([C:8]([CH2:9][CH3:10])=[O:11])([CH3:12])[CH2:4][C:5]1([CH3:6])[CH3:7].[OH2:13].O=CC2=CC(C(C)(C)C)=C(C=C2)C=O'),
 ('([CH-:1]-[C;H0;D3;+0:2]=[O:3].[O:4]=[C;H1;D2;+0:5][*;H0;+0:6])>>[O;-1:4]-[C;H1;D3;+0:5](-[*;H0;+0:6])-[CH;+0:1]-[C;H0;D3;+0:2]=[O:3]',
  '[O:0]=[C:1]1[CH-:2][C:3]([C:4]([CH2:5][CH3:6])=[O:7])([CH3:8])[CH2:9][C:10]1([CH3:11])[CH3:12].[O:13]=[CH:14][c:15]1[cH:16][c:17]([C:18]([CH3:19])([CH3:20])[CH3:21])[c:22]([CH:25]=[O:26])[cH:23][cH:24]1>[OH2:13]>[O:0]=[C:1]1[CH:2]([CH:25]([c:22]2[c:17]([C:18]([CH3:19])([CH3:20])[CH3:21])[cH:16][c:15]([CH:14]=[O:13])[cH:24][cH:23]2)[O-:26])[C:3]([C:4]([CH2:5][CH3:6])=[O:7])

In [11]:
param_list = ['([CH2:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH-:4])>>[CH-:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH2;+0:4]',
              '([CH-:1]-[C;H0;D3;+0:2]=[O:3].[O:4]=[C;H1;D2;+0:5][*;H0;+0:6])>>[O;-1:4]-[C;H1;D3;+0:5](-[*;H0;+0:6])-[CH;+0:1]-[C;H0;D3;+0:2]=[O:3]',
             '([O-:1].[OH2:2])>>[OH;+0:1].[OH-:2]',
             '([O:1]=[C;H0:2]-[C;H1:3])>>[O:1]=[C;H0:2]-[C;H0;-1:3]']


start = 'O=Cc1c(C)cc(C=O)cc1.CCC(C2CCC(C2)=O)=O.[OH-]'
target_output = 'CCC(C1CCC([C-]1C(O)c2c(C)cc(C=O)cc2)=O)=O'
output = dfs_with_processing1(start, transform_func, match_pdts, target_output, param_list)

Got it, matched!


In [12]:
mechanism = [output[i][1] for i in range(len(output))]
mechanism

['[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH2:7]1)=[O:9].[OH-:10]>O=Cc1c(C)cc(C=O)cc1>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH-:7]1)=[O:9].[OH2:10]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH-:8]1)=[O:9].[O:10]=[CH:11][c:12]1[c:13]([CH3:14])[cH:15][c:16]([CH:17]=[O:18])[cH:19][cH:20]1>[OH2:10]>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:11]([O-:10])[c:12]1[c:13]([CH3:14])[cH:15][c:16]([CH:17]=[O:18])[cH:19][cH:20]1)=[O:9]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([O-:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20].[OH2:21]>>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20].[OH-:21]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20]>[OH-:21]>[CH3:0][CH2

In [13]:
param_list = ['([CH2:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH-:4])>>[CH-:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH2;+0:4]',
              '([CH-:1]-[C;H0;D3;+0:2]=[O:3].[O:4]=[C;H1;D2;+0:5][*;H0;+0:6])>>[O;-1:4]-[C;H1;D3;+0:5](-[*;H0;+0:6])-[CH;+0:1]-[C;H0;D3;+0:2]=[O:3]',
             '([O-:1].[OH2:2])>>[OH;+0:1].[OH-:2]',
             '([O:1]=[C;H0:2]-[C;H1:3])>>[O:1]=[C;H0:2]-[C;H0;-1:3]']


start = 'O=Cc1c(C)cc(C=O)cc1.CCC(C2CCC(C2)=O)=O.[OH-]'
target_output = 'CCC(C1CCC([C-]1C(O)c2c(C)cc(C=O)cc2)=O)=O'
output = dfs_with_processing1(start, transform_func, match_pdts, target_output, param_list)
mechanism = [output[i][1] for i in range(len(output))]
mechanism

Got it, matched!


['[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH2:7]1)=[O:9].[OH-:10]>O=Cc1c(C)cc(C=O)cc1>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH-:7]1)=[O:9].[OH2:10]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH-:8]1)=[O:9].[O:10]=[CH:11][c:12]1[c:13]([CH3:14])[cH:15][c:16]([CH:17]=[O:18])[cH:19][cH:20]1>[OH2:10]>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:11]([O-:10])[c:12]1[c:13]([CH3:14])[cH:15][c:16]([CH:17]=[O:18])[cH:19][cH:20]1)=[O:9]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([O-:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20].[OH2:21]>>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20].[OH-:21]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20]>[OH-:21]>[CH3:0][CH2

In [14]:
# template_smarts_str = '([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3]'
# str_of_smiles = '[CH3:0][C-:1]([C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH2:8]1)=[O:9])[CH:10]([c:11]1[cH:12][c:13]([CH3:14])[c:15]([CH:16]=[O:17])[cH:18][cH:19]1)[OH:20].[OH-:21]'
# reactant_reagents = find_all_reac_reag_comb(template_smarts_str,str_of_smiles)
# reactants_smiles_list = reactant_reagents[0][0]
# reagents_smiles_list = reactant_reagents[0][1]

In [15]:
# rxn = AllChem.ReactionFromSmarts('([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3]')
# rxn

In [16]:
param_list = ['([CH2:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH-:4])>>[CH-:1]-[C;H0;D3;+0:2](=[O:3])-[*:5].[OH2;+0:4]',
              '([CH-:1]-[C;H0;D3;+0:2]=[O:3].[O:4]=[C;H1;D2;+0:5][*;H0;+0:6])>>[O;-1:4]-[C;H1;D3;+0:5](-[*;H0;+0:6])-[CH;+0:1]-[C;H0;D3;+0:2]=[O:3]',
             '([O-:1].[OH2:2])>>[OH;+0:1].[OH-:2]',
             '([O:1]=[C;H0:2]-[C;H1:3].[OH-:4])>>[O:1]=[C;H0:2]-[C;H0;-1:3].[OH2;+0:4]',
             '([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3]']


start = 'O=Cc1c(C)cc(C=O)cc1.CCC(C2CCC(C2)=O)=O.[OH-]'
target_output = 'CCC(C1CCC(C1=Cc2ccc(C=O)cc2C)=O)=O'
output = dfs_with_processing1(start, transform_func, match_pdts, target_output, param_list)
mechanism = [output[i][1] for i in range(len(output))]
mechanism

No valid reactant-reagent combinations found for: ([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3] with [CH3:0][CH:1]([C:2]([C-:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH2:8]1)=[O:9])[CH:10]([OH:11])[c:12]1[c:13]([CH3:14])[cH:15][c:16]([CH:17]=[O:18])[cH:19][cH:20]1.[OH2:21]
No valid reactant-reagent combinations found for: ([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3] with [CH3:0][CH:1]([C:2]([C-:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH2:8]1)=[O:9])[CH:10]([c:11]1[cH:12][c:13]([CH3:14])[c:15]([CH:16]=[O:17])[cH:18][cH:19]1)[OH:20].[OH2:21]
No valid reactant-reagent combinations found for: ([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3] with [CH3:0][CH2:1][C:2]([C-:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([c:10]1[cH:11][c:12]([CH3:13])[c:14]([CH:15]=[O:16])[cH:17][cH:18]1)[OH:19])=[O:20].[OH2:21]
No valid reactant-reagent combinations found for: ([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3] with [CH3:0][CH2:1][C:2]([C-:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[c:12]([CH3:13])[c

['[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH2:7]1)=[O:9].[OH-:10]>O=Cc1c(C)cc(C=O)cc1>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH-:7]1)=[O:9].[OH2:10]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH-:8]1)=[O:9].[O:10]=[CH:11][c:12]1[c:13]([CH3:14])[cH:15][c:16]([CH:17]=[O:18])[cH:19][cH:20]1>[OH2:10]>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:11]([O-:10])[c:12]1[c:13]([CH3:14])[cH:15][c:16]([CH:17]=[O:18])[cH:19][cH:20]1)=[O:9]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([O-:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20].[OH2:21]>>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20].[OH-:21]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[c:12]([CH3:13])[cH:14][c:15]([CH:16]=[O:17])[cH:18][cH:19]1)=[O:20].[OH-:21]>>[CH3:0][CH

In [17]:
#test case1; 
start = 'CCC(C1CCC(C1)=O)=O.O=Cc2ccc(C=O)cc2Br.[OH-]'
target_output = 'CCC(C1CCC(C1=Cc2ccc(C=O)cc2Br)=O)=O' #selectivity: ortho to Bromine coupling
output = dfs_with_processing1(start, transform_func, match_pdts, target_output, param_list)
mechanism = [output[i][1] for i in range(len(output))]
mechanism

No valid reactant-reagent combinations found for: ([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3] with [CH3:0][CH2:1][C:2]([C-:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([c:10]1[cH:11][cH:12][c:13]([CH:14]=[O:15])[c:16]([Br:17])[cH:18]1)[OH:19])=[O:20].[OH2:21]
Got it, matched!


['[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH2:7]1)=[O:9].[OH-:10]>O=Cc2ccc(C=O)cc2Br>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH-:7]1)=[O:9].[OH2:10]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH-:8]1)=[O:9].[O:10]=[CH:11][c:12]1[cH:13][cH:14][c:15]([CH:16]=[O:17])[cH:18][c:19]1[Br:20]>[OH2:10]>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:11]([O-:10])[c:12]1[cH:13][cH:14][c:15]([CH:16]=[O:17])[cH:18][c:19]1[Br:20])=[O:9]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([O-:10])[c:11]1[cH:12][cH:13][c:14]([CH:15]=[O:16])[cH:17][c:18]1[Br:19])=[O:20].[OH2:21]>>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[cH:12][cH:13][c:14]([CH:15]=[O:16])[cH:17][c:18]1[Br:19])=[O:20].[OH-:21]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([OH:10])[c:11]1[cH:12][cH:13][c:14]([CH:15]=[O:16])[cH:17][c:18]1[Br:19])=[O:20].[OH-:21]>>[CH3:0][CH2:1][C:2]([CH:3]

In [18]:
#test case2; 
start = 'CCC(C1CCC(C1)=O)=O.O=Cc2ccc(C=O)cc2Br.[OH-]'
target_output = 'CCC(C1CCC(/C1=C\c2cc(Br)c(C=O)cc2)=O)=O' #selectivity: meta to Bromine coupling
output = dfs_with_processing1(start, transform_func, match_pdts, target_output, param_list)
mechanism = [output[i][1] for i in range(len(output))]
mechanism

No valid reactant-reagent combinations found for: ([C;-1:1]-[C:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3] with [CH3:0][CH2:1][C:2]([C-:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([c:10]1[cH:11][cH:12][c:13]([CH:14]=[O:15])[c:16]([Br:17])[cH:18]1)[OH:19])=[O:20].[OH2:21]
Got it, matched!


['[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH2:7]1)=[O:9].[OH-:10]>O=Cc2ccc(C=O)cc2Br>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:8])[CH-:7]1)=[O:9].[OH2:10]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH-:8]1)=[O:9].[O:10]=[CH:11][c:12]1[cH:13][cH:14][c:15]([CH:16]=[O:17])[cH:18][c:19]1[Br:20]>[OH2:10]>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:16]([c:15]1[cH:14][cH:13][c:12]([CH:11]=[O:10])[c:19]([Br:20])[cH:18]1)[O-:17])=[O:9]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([c:10]1[cH:11][cH:12][c:13]([CH:14]=[O:15])[c:16]([Br:17])[cH:18]1)[O-:19])=[O:20].[OH2:21]>>[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([c:10]1[cH:11][cH:12][c:13]([CH:14]=[O:15])[c:16]([Br:17])[cH:18]1)[OH:19])=[O:20].[OH-:21]',
 '[CH3:0][CH2:1][C:2]([CH:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH:8]1[CH:9]([c:10]1[cH:11][cH:12][c:13]([CH:14]=[O:15])[c:16]([Br:17])[cH:18]1)[OH:19])=[O:20].[OH-:21]>>[CH3:0][CH2:1][C:2

In [19]:
# smiles = '[CH3:0][CH:1]([C:2]([C-:3]1[CH2:4][CH2:5][C:6](=[O:7])[CH2:8]1)=[O:9])[CH:10]([OH:11])[c:12]1[c:13]([CH3:14])[cH:15][c:16]([CH:17]=[O:18])[cH:19][cH:20]1.[OH-:21]'
# temp = '([*;-1:1]-[*:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3]'
# reactant_reagents = find_all_reac_reag_comb(temp,smiles)
# reactant_reagents

In [21]:
# smiles = 'C[C-](C(c1ccc(C=O)cc1C)O)C(C2CCC(C2)=O)=O.[OH-]'
# temp = '([*;-1:1]-[*:2]-[OH:3])>>[C;+0:1]=[C:2].[OH-:3]'
# reactant_reagents = find_all_reac_reag_comb(temp,smiles)
# reactant_reagents